In [1]:
# Importiamo le librerie

from itertools import combinations
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from itertools import combinations
from os import listdir
from os.path import isfile, join
import numpy as np

In [2]:
# Variabili globali

mypath = 'dataset'

# Mi vado a prendere i path di tutti i file nella cartella dataset

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Mi vado a generare tutte le possibili combinazioni di due elementi da un insieme di 5 elementi


comb = list(combinations(range(0,5), 2))

In [3]:
print(onlyfiles)

['Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv', 'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv', 'Friday-02-03-2018_TrafficForML_CICFlowMeter.csv', 'Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv', 'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv', 'Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv', 'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv', 'Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv', 'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv', 'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv']


In [4]:

document = 'Wednesday-28-02-2018_TrafficForML_CICFlowMeter'

In [5]:
# Andiamo a pulire tutti i dataset

# Come prima cosa andiamo a leggere le feature dai vari file csv

anova = pd.read_csv('top_feature/'+document+"/anova.csv")
tree = pd.read_csv('top_feature/'+document+"/ExtraTree.csv")
svm = pd.read_csv('top_feature/'+document+"/SVM.csv")
rfe = pd.read_csv('top_feature/'+document+"/RFE.csv")


In [6]:
# Andiamo a normalizzare tutti gli score delle feature, in maniera tale da avere valori compresi tra 0 e 1

anova['F_Score'] = anova['F_Score']/anova['F_Score'].max()
tree['F_Score'] = tree['F_Score']/tree['F_Score'].max()
svm['F_Score'] = svm['F_Score']/svm['F_Score'].max()
rfe['F_Score'] = rfe['F_Score']/rfe['F_Score'].max()

In [7]:
display(svm)

,F_Score,Input_Features
0,1.000000,Init Bwd Win Byts
1,0.706251,Flow IAT Std
2,0.641805,Bwd Pkt Len Max
3,0.640393,Pkt Len Max
4,0.562364,Fwd IAT Std
5,0.485466,Flow IAT Mean
6,0.480161,Bwd Seg Size Avg
7,0.480161,Bwd Pkt Len Mean
8,0.437510,Fwd IAT Mean
9,0.418868,Pkt Len Mean


In [8]:
# Facciamo una concatenazione dei vari dataframe.
# Saranno utilizzati per prendere le feature che si ripetono di più e che hanno i punteggi maggiori

total_feature = pd.concat([anova,tree,svm,rfe],ignore_index=True)
total_feature_count = pd.concat([anova,tree,svm,rfe],ignore_index=True)

In [9]:
# Andiamo a eseguire un operazione di group by, con successivo count delle occorrenze dei vari campi

total_feature_count = total_feature_count.groupby(['Input_Features']).count().reset_index().sort_values(by='F_Score',ascending=False)

# Andiamo a rinominare la colonna F_Score, che adesso contiene i valori di count

total_feature_count.rename(columns = {'F_Score':'Conto'}, inplace = True)
print(total_feature_count)

       Input_Features  Conto
9       Flow IAT Mean      3
11        Flow Pkts/s      3
0         Bwd IAT Tot      2
21  Init Fwd Win Byts      2
20  Init Bwd Win Byts      2
17         Fwd Pkts/s      2
13        Fwd IAT Std      2
10       Flow IAT Std      2
12       Fwd IAT Mean      2
8        Flow IAT Max      2
7       Flow Duration      2
6         Flow Byts/s      2
3          Bwd Pkts/s      2
1     Bwd Pkt Len Max      1
5      CWE Flag Count      1
14        Fwd IAT Tot      1
15    Fwd Pkt Len Max      1
16   Fwd Pkt Len Mean      1
4    Bwd Seg Size Avg      1
18   Fwd Seg Size Avg      1
19      Fwd URG Flags      1
2    Bwd Pkt Len Mean      1
22        Pkt Len Max      1
23       Pkt Len Mean      1
24       Pkt Size Avg      1


In [10]:
# Delle delle feature più frequenti mi interessano le prime dieci

most_frequent = total_feature_count[:10]

print(most_frequent)

       Input_Features  Conto
9       Flow IAT Mean      3
11        Flow Pkts/s      3
0         Bwd IAT Tot      2
21  Init Fwd Win Byts      2
20  Init Bwd Win Byts      2
17         Fwd Pkts/s      2
13        Fwd IAT Std      2
10       Flow IAT Std      2
12       Fwd IAT Mean      2
8        Flow IAT Max      2


In [11]:
# Ci serve ancora il parametro F_Scores, quindi creiamo un nuovo dataframe con queste righe.
# Abbiamo le dieci righe, però, sono ancora presenti le ridondanze che prima avevamo eliminato.

test = pd.DataFrame(data=[],columns=total_feature.columns)

for a in most_frequent['Input_Features']:
    test = pd.concat([test,total_feature[total_feature['Input_Features'] == a]])

print(test)

       Input_Features   F_Score
18      Flow IAT Mean  0.715298
25      Flow IAT Mean  0.485466
36      Flow IAT Mean  1.000000
1         Flow Pkts/s  0.459988
10        Flow Pkts/s  1.000000
35        Flow Pkts/s  1.000000
19        Bwd IAT Tot  0.683407
31        Bwd IAT Tot  1.000000
6   Init Fwd Win Byts  0.144963
13  Init Fwd Win Byts  0.886946
2   Init Bwd Win Byts  0.246871
20  Init Bwd Win Byts  1.000000
3          Fwd Pkts/s  0.160852
12         Fwd Pkts/s  0.965384
24        Fwd IAT Std  0.562364
34        Fwd IAT Std  1.000000
21       Flow IAT Std  0.706251
37       Flow IAT Std  1.000000
28       Fwd IAT Mean  0.437510
30       Fwd IAT Mean  1.000000
11       Flow IAT Max  0.989897
38       Flow IAT Max  1.000000


In [12]:
# Creo una variabile all'interno del quale vado a salvare il groupby del precedente dataframe, solo che in uesto caso
# andiamo a sommare i punteggi di F_Scores, invece di contare le occorrenze.

group_by = test.groupby(['Input_Features'])['F_Score'].sum().reset_index()

In [13]:
print(group_by)

      Input_Features   F_Score
0        Bwd IAT Tot  1.683407
1       Flow IAT Max  1.989897
2      Flow IAT Mean  2.200764
3       Flow IAT Std  1.706251
4        Flow Pkts/s  2.459988
5       Fwd IAT Mean  1.437510
6        Fwd IAT Std  1.562364
7         Fwd Pkts/s  1.126236
8  Init Bwd Win Byts  1.246871
9  Init Fwd Win Byts  1.031909


In [14]:
# Facciamo un ordinamento del dataframe precedente

print(group_by.sort_values(by='F_Score',ascending=False))

      Input_Features   F_Score
4        Flow Pkts/s  2.459988
2      Flow IAT Mean  2.200764
1       Flow IAT Max  1.989897
3       Flow IAT Std  1.706251
0        Bwd IAT Tot  1.683407
6        Fwd IAT Std  1.562364
5       Fwd IAT Mean  1.437510
8  Init Bwd Win Byts  1.246871
7         Fwd Pkts/s  1.126236
9  Init Fwd Win Byts  1.031909


In [15]:
# A questo punto, terminiamo l'operazione andando a moltiplicare la frequenza per la somma dei punteggi delle varie feature.

last_feature = pd.DataFrame(data=[],columns=group_by.columns)

for a,b in zip(most_frequent['Input_Features'],most_frequent['Conto']):
    last_feature.loc[len(last_feature)] =  {'Input_Features':a,'F_Score':group_by[group_by['Input_Features'] == a]['F_Score'].values[0]*b}
    

In [16]:
# Facciamo una stampa ordinata

print(last_feature.sort_values(by='F_Score',ascending=False))

      Input_Features   F_Score
1        Flow Pkts/s  7.379965
0      Flow IAT Mean  6.602293
9       Flow IAT Max  3.979795
7       Flow IAT Std  3.412502
2        Bwd IAT Tot  3.366814
6        Fwd IAT Std  3.124727
8       Fwd IAT Mean  2.875020
4  Init Bwd Win Byts  2.493741
5         Fwd Pkts/s  2.252472
3  Init Fwd Win Byts  2.063818


In [17]:

# A questo punto ci interessano solo le prime 5 feature, e tutte le loro possibili combinazioni di due elementi.

esito_finale = pd.DataFrame(data=[],columns=['Feature_1','Feature_2'])
top_index = []

# Qui andiamo a prendere gli indici delle prime 5 features

for a in range(0,5):
    top_index.append(last_feature.sort_values(by='F_Score',ascending=False).index[a])
    

# Andiamo a inserire tutte le possibili combinazioni nel dataframe

for b in comb:
    print(last_feature.loc[top_index[b[0]]]['Input_Features'])
    print(last_feature.loc[top_index[b[1]]]['Input_Features'])
    esito_finale.loc[len(esito_finale)] =  {
        'Feature_1': last_feature.loc[top_index[b[0]]]['Input_Features'],
        'Feature_2': last_feature.loc[top_index[b[1]]]['Input_Features']
        }
    

Flow Pkts/s
Flow IAT Mean
Flow Pkts/s
Flow IAT Max
Flow Pkts/s
Flow IAT Std
Flow Pkts/s
Bwd IAT Tot
Flow IAT Mean
Flow IAT Max
Flow IAT Mean
Flow IAT Std
Flow IAT Mean
Bwd IAT Tot
Flow IAT Max
Flow IAT Std
Flow IAT Max
Bwd IAT Tot
Flow IAT Std
Bwd IAT Tot


In [18]:
print(esito_finale)

       Feature_1      Feature_2
0    Flow Pkts/s  Flow IAT Mean
1    Flow Pkts/s   Flow IAT Max
2    Flow Pkts/s   Flow IAT Std
3    Flow Pkts/s    Bwd IAT Tot
4  Flow IAT Mean   Flow IAT Max
5  Flow IAT Mean   Flow IAT Std
6  Flow IAT Mean    Bwd IAT Tot
7   Flow IAT Max   Flow IAT Std
8   Flow IAT Max    Bwd IAT Tot
9   Flow IAT Std    Bwd IAT Tot


In [19]:
top5 = esito_finale.Feature_1.unique()
top5 = np.append(top5,esito_finale.Feature_2.unique())
top5 = pd.Series(top5)
top5 = top5.unique()
top5 = pd.Series(top5)
print(top5)

top5.to_csv('top_feature/'+document+'/top5.csv',index=False)

0      Flow Pkts/s
1    Flow IAT Mean
2     Flow IAT Max
3     Flow IAT Std
4      Bwd IAT Tot
dtype: object


In [20]:
esito_finale.to_csv('top_feature/'+document+'/combinazioni.csv',index=False,header=False)